In [7]:
import json


def nlu_wrapper(text, url='http://172.20.78.250:30000/'):
    """분석후 결과를 반환한다."""
    import requests
    from pprint import pprint

    req_data = {
        "nlu_wrapper": {
            "option": {
                "domain": "baseball",
                "style": "literary",
                "module": ["SBD_crf", "POS", "NER", "DP", "TIME_python"],
            }
        },
        "doc": [{
            "contents": text
        }]
    }

    meta = None
    result = []
    try:
        resp = requests.post(url=url, json=req_data, timeout=10)

        nlu_resp = resp.json()
        if 'nlu_wrapper' in nlu_resp:
            meta = nlu_resp['nlu_wrapper']

        # 단순화시킨다.
        for doc in nlu_resp['doc']:
            for sentence in doc['sentences']:
                #                 pprint(sentence)

                item = {
                    'text': '',
                    'morp_str': '',
                    'ne_str': '',
                    'depen_str': [],
                    'time_results': [],
                }

                for k in item.keys():
                    if k not in sentence:
                        continue

                    item[k] = sentence[k]
                    
                # 시간 필드 변경
                if len(item['time_results']) > 0:
                    item['times'] = item['time_results']
                    
                del item['time_results']

                # depen_str 변환
                if len(item['depen_str']) > 0:
                    depen = []
                    for dep in item['depen_str']:
                        depen.append(dict(zip(['index', 'head', 'morp', 'pos', 'func'], dep)))
                        
                    item['depen'] = depen
                    
                del item['depen_str']

                result.append(item)

        return result, meta
    except Exception as e:
        print(e)

    return result, meta


text = '''
지난해 출생아 수가 30만명대에 턱걸이하면서 합계출산율이 사상 최저인 0.98명으로 떨어졌다. 청년층의 결혼 기피 현상이 심화하는 데다 기혼 여성의 출산 연령이 높아지면서 출산율이 급락하고 있다. 올해는 상황이 더욱 심각하다. 2분기 합계출산율 잠정치는 0.91명으로 곤두박질했다. 올해 출생아 수는 30만명 선마저 붕괴할 것이 확실시된다.

통계청이 28일 발표한 2018년 출생 통계(확정)를 보면 지난해 출생아 수는 32만6800명으로 집계됐다. 1년 전보다 8.7% 줄어든 수치로, 1970년 관련 통계 작성 이래 최저다.

합계출산율은 사상 처음 1.0명 선이 무너졌다. 합계출산율은 여성 1명이 평생 낳을 것으로 예상되는 평균 출생아 수를 뜻한다. 인구 유지를 위해 필요한 합계출산율이 2.1명인 점을 감안하면, 절반에도 미치지 못한다는 얘기다. 경제협력개발기구(OECD) 36개 회원국의 평균(2017년 기준 1.65명)에 크게 미달할 뿐 아니라 맨 꼴찌다.
'''

text = "NC 다이노스의 테임즈는 오늘 시즌 10호 홈런을 쳤다."
text = "85년생 35살 입니다."

text = "[OSEN=기장(부산)박준형 기자] 30일 오후 부산 기장군 기장현대차드림볼파크에서 제29회 WBSC 기장 세계청소년야구선수권대회(18세 이하)' 일본과 스페인의 경기가 진행됐다.\n\n7회말 실점위기를 무실점으로 막은 스페인 선발투수 저스틴 루나가 동료들과 하이파이브를 하고 있다. / soul1014@osen.co.kr"

r = nlu_wrapper(text=text)

print(json.dumps(r, ensure_ascii=False, indent=4))

[
    [
        {
            "text": "[OSEN=기장(부산)박준형 기자] 30일 오후 부산 기장군 기장현대차드림볼파크에서 제29회 WBSC 기장 세계청소년야구선수권대회(18세 이하)' 일본과 스페인의 경기가 진행됐다. 7회말 실점위기를 무실점으로 막은 스페인 선발투수 저스틴 루나가 동료들과 하이파이브를 하고 있다.",
            "morp_str": "[/SS+OSEN/SL+=/SW+기장/NNP+(/SS+부산/NNP+)/SS+박준형/NNP 기자/NNG+]/SS 30/SN+일/NNB 오후/NNG 부산/NNP 기장군/NNP 기장/NNP+현대차/NNP+드림/NNG+볼/NNG+파크/NNG+에서/JKB 제/MM+29/SN+회/NNB WBSC/SL 기장/NNG 세계/NNG+청소년/NNG+야구/NNG+선수권/NNG+대회/NNG+(/SS+18/SN+세/NNB 이하/NNG+)/SS+'/SS 일본/NNP+과/JKB 스페인/NNP+의/JKG 경기/NNG+가/JKS 진행/NNG+되/XSV+었/EP+다/EM+./SF 7/SN+회/NNB+말/NNB 실점/NNG+위기/NNG+를/JKO 무실점/NNG+으로/JKB 막/VV+은/ETM 스페인/NNP 선발/NNG+투수/NNG 저스틴/NNP 루나/NNP+가/JKS 동료/NNG+들/XSN+과/JKB 하이파이브/NNG+를/JKO 하/VV+고/EM 있/VX+다/EM+./SF",
            "ne_str": "[OSEN=기장(<부산:LOC_CIT>)<박준형:PER_OPE> 기자] <30일:DATE_ETC> <오후:TIME> <부산:LOC_CIT> <기장군:LOC_ETC> <기장현대차드림볼파크:LOC_STA>에서 <제29회:QT_ORDER> WBSC 기장 세계청소년<야구:TM_BB_ETC>선수권대회(<18세:QT_AGE> 이하)' <일본:LOC_NAT>과 스페인의 경기가 진행됐다. <7회말:QT_ORDER> <실점:TM_SPORTS_RECORD>위기를 <무실점:TM_SPORTS_RECORD>으로 막은